In [ ]:
import pickle
import numpy
import pandas as pd
from os import path

state_gov_nm = 'CA'
year_to_process = '2018'
data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

state_gov_nm = sys.argv[1]
year_to_process = sys.argv[2]
data_file_location = sys.argv[3]

In [ ]:
leg_data = pd.DataFrame()
legcorpus = []
win_state = []
_legtext = path.join(data_file_location,state_gov_nm + year_to_process + 'embedded_legdoc.df')
_bill_clusters = path.join(data_file_location,state_gov_nm + year_to_process + 'bill_clusters.df')
_cmltv_event_df = path.join(data_file_location,state_gov_nm + 'cmltv_events.df')
#_sponsors = path.join(data_file_location, state_gov_nm + 'Sponsors.csv')

In [ ]:
for i in range(2009, int(year_to_process)+1):

    try:
        _WinList = path.join(data_file_location, state_gov_nm + str(i) + 'WinList.list')
        
        _LegOverallData_CSV = path.join(data_file_location, state_gov_nm + str(i) + 'LegOverallData.csv')
        
        leg_data = leg_data.append(pd.read_csv(_LegOverallData_CSV))


        with open(_WinList,'rb') as f:
            win_state.extend(pickle.load(f))

    except:
        continue

with open(_legtext,'rb') as f:
    legtextembed = pickle.load(f)

with open(_bill_clusters,'rb') as f:
    billcat = pickle.load(f)

#leg_data = leg_data.join(billcat, how='inner', on='bill_id')

#sponsor_df = pd.read_csv(_sponsors)
#sponsor_df.set_index('bill_id', in_place = True)

In [ ]:
#create a cumulative event list for each time_t
#also creation of a column to determine when a pass or fail occurs, vs a regular move (can expand on this later on)
e_hist_list = []
prev_bill_id = 0
event_hist = ''
move_list = []
first = True
#iterate through the rows of the dataframe
for r in leg_data.iterrows():
    if r[1]['bill_id'] == prev_bill_id:
        event_hist = event_hist + str(r[1]['event']) + '->'
        e_hist_list.append(event_hist)
        if 'adopted' in event_hist or 'chaptered' in event_hist:
            move_list.append('Pass')
        else:
            move_list.append('Move')
    else:
        event_hist = r[1]['event'] + '->'
        e_hist_list.append(event_hist)
        if first:
            first = False
        elif move_list[-1] != 'Pass':
            move_list[-1] = 'Fail'
        
        move_list.append('Move')
        
    prev_bill_id = r[1]['bill_id']

In [ ]:
#add the new lists as columns in the dataframe
leg_data['cmltv_event'] = e_hist_list
leg_data['move'] = move_list

In [ ]:
#split the training and test data sets
leg_train = leg_data.where(leg_data['year'] < int(year_to_process) & leg_data['bill_cat']=='18', inplace = False)
leg_test = leg_data.where(leg_data['year'] == int(year_to_process) & leg_data['bill_cat']=='18', inplace = False)

In [ ]:
leg_to_export = leg_data[['bill_id','time_t','cmltv_event','move']].copy()

with open(_cmltv_event_df,'wb') as f:
    pickle.dump(leg_to_export, f)

In [ ]:
#leg_event_counts = leg_train.groupby(['bill_cat','cmltv_event'])['bill_id'].count()
#leg_counts = leg_train.groupby('bill_cat')['bill_id'].count()

In [ ]:
#comb = pd.merge(leg_counts,leg_event_counts,on='bill_cat',how='right')

In [ ]:
#comb

In [ ]:
#leg_data.head()